In [1]:
from mlx_utils import build_model, mlx_encode

/Users/rickyhsu/miniconda3/envs/rag-label/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, tokenizer = build_model('model_cfg.json', weight_path='mlx_bert.npz')
texts = ['今天天氣如何']
seq_embed, pooled_embed, attn_mask = mlx_encode(texts[0], tokenizer, model)

In [3]:
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F
from torch import Tensor
def average_pool(last_hidden_states: Tensor,
    attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
model_name = 'intfloat/multilingual-e5-small'
max_length = 512
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cpu')
model.eval()
batch_dict = tokenizer([texts[0]], max_length=512, padding=True, truncation=True, return_tensors='pt')
outputs = model(**batch_dict)
# print('outputs')
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1).tolist()

In [19]:
import torch
mlx_embed = torch.tensor(seq_embed)
mlx_embed = average_pool(mlx_embed, torch.tensor(attn_mask))
mlx_embed = F.normalize(mlx_embed, p=2, dim=1).tolist()

In [7]:
tmp

[0.04776264727115631,
 [0.04776265099644661,
  0.005975227802991867,
  -0.04697135463356972,
  -0.0736924409866333,
  0.07657074183225632,
  0.014487102627754211,
  -0.02469732239842415,
  0.04852293059229851,
  0.06825117766857147,
  -0.011381318792700768,
  0.040964920073747635,
  0.010725769214332104,
  0.05205019935965538,
  -0.03589736670255661,
  -0.0935419574379921,
  0.03976619988679886,
  0.05796697363257408,
  -0.039498671889305115,
  0.005081417970359325,
  -0.05264614149928093,
  0.02767621912062168,
  0.01664052903652191,
  -0.07125937938690186,
  0.03156953677535057,
  0.06225759536027908,
  0.03824276104569435,
  -0.022160453721880913,
  0.0382501557469368,
  0.04634709283709526,
  -0.05332187935709953,
  -0.011784614063799381,
  -0.06997958570718765,
  0.10343431681394577,
  -0.04151371493935585,
  0.07515836507081985,
  0.049190085381269455,
  -0.00945397187024355,
  -0.06321169435977936,
  0.046696241945028305,
  -0.08927010744810104,
  -0.001231140224263072,
  0.0411

In [23]:
from sentence_transformers import util
tmp = [mlx_embed[0], embeddings[0]]
util.cos_sim(tmp, tmp)

tensor([[1.0000, 1.0000],
        [1.0000, 1.0000]])